In [1]:
%pylab inline
from ipyparallel import Client, error
cluster=Client(profile="mpi") 
view=cluster[:]
view.block=True

Populating the interactive namespace from numpy and matplotlib


/Users/dkilkenny/anaconda3/envs/openmdao_38/lib/python3.8/site-packages/ipyparallel/util.py:213: RuntimeWarning: IPython could not determine IPs for GRSLA17060172: [Errno 8] nodename nor servname provided, or not known
  warnings.warn("IPython could not determine IPs for %s: %s" % (host, e),


In [2]:
try:
    import openmdao.api as om
except ImportError:
    !python -m pip install openmdao[notebooks]
    import openmdao.api as om

# Using src_indices with Promoted Variables
---

Inputs and outputs can be connected by promoting them both to the same name, but what
if your output is an array and you only want to connect part of it to your
input?  

If you connect variables via promotion, you must set `src_indices` either
when you call `add_input` to add your input variable to its component or 
when you promote the input with the :ref:`promotes <group_promotes>` method.  Another
argument, `flat_src_indices` is a boolean that determines whether the entries
of the `src_indices` array are interpreted as indices into the flattened source
or as tuples or lists of indices into the unflattened source.  The default
of `flat_src_indices=False` assumes indices map into an unflattened source.

Usage
-----

1: Connect an independent array variable to two different components where each component gets part of the array.  

In [3]:
import numpy as np 

class MyComp1(om.ExplicitComponent):
    def setup(self):
        # this input will connect to entries 0, 1, and 2 of its source
        self.add_input('x', np.ones(3), src_indices=[0, 1, 2])
        self.add_output('y', 1.0)

    def compute(self, inputs, outputs):
        outputs['y'] = np.sum(inputs['x'])*2.0

class MyComp2(om.ExplicitComponent):
    def setup(self):
        # this input will connect to entries 3 and 4 of its source
        self.add_input('x', np.ones(2), src_indices=[3, 4])
        self.add_output('y', 1.0)

    def compute(self, inputs, outputs):
        outputs['y'] = np.sum(inputs['x'])*4.0

p = om.Problem()

# by promoting the following output and inputs to 'x', they will
# be automatically connected
p.model.add_subsystem('indep', om.IndepVarComp('x', np.ones(5)),
                      promotes_outputs=['x'])
p.model.add_subsystem('C1', MyComp1(), promotes_inputs=['x'])
p.model.add_subsystem('C2', MyComp2(), promotes_inputs=['x'])

p.setup()
p.run_model()

print(p.get_val('C1.x'))
print(p.get_val('C1.y'))
print(p.get_val('C2.x'))
print(p.get_val('C2.y'))

[1. 1. 1.]
[6.]
[1. 1.]
[8.]


/Users/dkilkenny/Desktop/OpenMDAO_Clone/openmdao/core/component.py:514: DeprecationWarning:'C1' <class MyComp1>: Passing `src_indices` as an arg to `add_input` isdeprecated and will become an error in a future release.  Add `src_indices` to a `promotes` or `connect` call instead.
/Users/dkilkenny/Desktop/OpenMDAO_Clone/openmdao/core/component.py:514: DeprecationWarning:'C2' <class MyComp2>: Passing `src_indices` as an arg to `add_input` isdeprecated and will become an error in a future release.  Add `src_indices` to a `promotes` or `connect` call instead.


In [4]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(p.get_val('C1.x'), np.ones(3))
assert_near_equal(p.get_val('C1.y'), 6.)
assert_near_equal(p.get_val('C2.x'), np.ones(2))
assert_near_equal(p.get_val('C2.y'), 8.)

0.0

2: A distributed component that promotes its input and receives certain entries of the source array based on its rank.  Note that negative indices are supported.

In [12]:
%%px

import openmdao.api as om
import numpy as np

class MyComp(om.ExplicitComponent):
    def setup(self):
        # decide what parts of the array we want based on our rank
        if self.comm.rank == 0:
            idxs = [0, 1, 2]
        else:
            # use [3, -1] here rather than [3, 4] just to show that we
            # can use negative indices.
            idxs = [3, -1]

        self.add_input('x', np.ones(len(idxs)), src_indices=idxs)
        self.add_output('y', 1.0)

    def compute(self, inputs, outputs):
        outputs['y'] = np.sum(inputs['x'])*2.0

p = om.Problem()

p.model.add_subsystem('indep', om.IndepVarComp('x', np.arange(5, dtype=float)),
                      promotes_outputs=['x'])

p.model.add_subsystem('C1', MyComp(),
                      promotes_inputs=['x'])

p.setup()
p.set_val('x', np.arange(5, dtype=float))
p.run_model()

# each rank holds the assigned portion of the input array
print(p.get_val('C1.x'))

[stdout:0] [0. 1. 2.]
[stdout:1] [3. 4.]
[stdout:2] [3. 4.]
[stdout:3] [3. 4.]


[stderr:0] /Users/dkilkenny/Desktop/OpenMDAO_Clone/openmdao/core/component.py:514: DeprecationWarning:'C1' <class MyComp>: Passing `src_indices` as an arg to `add_input` isdeprecated and will become an error in a future release.  Add `src_indices` to a `promotes` or `connect` call instead.
[stderr:1] /Users/dkilkenny/Desktop/OpenMDAO_Clone/openmdao/core/component.py:514: DeprecationWarning:'C1' <class MyComp>: Passing `src_indices` as an arg to `add_input` isdeprecated and will become an error in a future release.  Add `src_indices` to a `promotes` or `connect` call instead.
[stderr:2] /Users/dkilkenny/Desktop/OpenMDAO_Clone/openmdao/core/component.py:514: DeprecationWarning:'C1' <class MyComp>: Passing `src_indices` as an arg to `add_input` isdeprecated and will become an error in a future release.  Add `src_indices` to a `promotes` or `connect` call instead.
[stderr:3] /Users/dkilkenny/Desktop/OpenMDAO_Clone/openmdao/core/component.py:514: DeprecationWarning:'C1' <class MyComp>: Pass

In [13]:
# the output in each rank is based on the local inputs
print(p.get_val('C1.y'))

[6.]


In [21]:
%%px
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(p['C1.x'],
                  np.arange(3, dtype=float) if p.model.C1.comm.rank == 0 else np.arange(3, 5, dtype=float))

# the output in each rank is based on the local inputs
assert_near_equal(p['C1.y'], 6. if p.model.C1.comm.rank == 0 else 14.)

Out[0:7]: 0.0

Out[1:7]: 0.0

Out[2:7]: 0.0

Out[3:7]: 0.0

3: The source array is shape (4,3) and the input array is shape (2,2)


In [22]:
class MyComp(om.ExplicitComponent):
    def setup(self):
        # We want to pull the following 4 values out of the source:
        # [(0,0), (3,1), (2,1), (1,1)].
        # Because our input is also non-flat we arrange the
        # source index tuples into an array having the same shape
        # as our input.  If we didn't set flat_src_indices to False,
        # we could specify src_indices as a 1D array of indices into
        # the flattened source.
        self.add_input('x', np.ones((2, 2)),
                       src_indices=[[(0, 0), (3, 1)],
                                    [(2, 1), (1, 1)]],
                       flat_src_indices=False)
        self.add_output('y', 1.0)

    def compute(self, inputs, outputs):
        outputs['y'] = np.sum(inputs['x'])

p = om.Problem()

# by promoting the following output and inputs to 'x', they will
# be automatically connected
p.model.add_subsystem('indep',
                      om.IndepVarComp('x', np.arange(12).reshape((4, 3))),
                      promotes_outputs=['x'])
p.model.add_subsystem('C1', MyComp(),
                      promotes_inputs=['x'])

p.setup()
p.run_model()

print(p.get_val('C1.x'))
print(p.get_val('C1.y'))

[[ 0. 10.]
 [ 7.  4.]]
[21.]


/Users/dkilkenny/Desktop/OpenMDAO_Clone/openmdao/core/component.py:514: DeprecationWarning:'C1' <class MyComp>: Passing `src_indices` as an arg to `add_input` isdeprecated and will become an error in a future release.  Add `src_indices` to a `promotes` or `connect` call instead.


In [23]:
assert_near_equal(p.get_val('C1.x'),
                 np.array([[0., 10.],
                           [7., 4.]]))
assert_near_equal(p.get_val('C1.y'), 21.)

0.0

4: If the source array is shape (4,3), the input is scalar, and we want to
connect it to the (3, 1) entry of the source, then the `add_input`
call might look like the following if we use `flat_src_indices`:

```
    self.add_input('x', 1.0, src_indices=[10], shape=1, flat_src_indices=True)
```

If we instead use the default setting of `flat_src_indices=False`, we must specify
the input shape since a scalar input value alone doesn't necessarily indicate that
the input variable is scalar.  For example, in the case below, if we didn't know the
input shape, we wouldn't know if it was scalar and connected to a 2-D source
array, or if it was shape (1,2) and connected to a flat source array.

```
    self.add_input('x', 1.0, src_indices=np.array([[3,1]]), shape=1)
```

5: If the source array is flat and the input is shape (2,2), the `add_input` call might look like this:
```
    self.add_input('x', src_indices=[[0, 10], [7, 4]], shape=(2,2))
```

6: Indices can also be specified when an input is promoted via the :ref:`promotes <group_promotes>` 
method.

In [24]:
class MyComp1(om.ExplicitComponent):
    """ multiplies input array by 2. """
    def setup(self):
        self.add_input('x', np.ones(3))
        self.add_output('y', 1.0)

    def compute(self, inputs, outputs):
        outputs['y'] = np.sum(inputs['x'])*2.0

class MyComp2(om.ExplicitComponent):
    """ multiplies input array by 4. """
    def setup(self):
        self.add_input('x', np.ones(2))
        self.add_output('y', 1.0)

    def compute(self, inputs, outputs):
        outputs['y'] = np.sum(inputs['x'])*4.0

class MyGroup(om.Group):
    def setup(self):
        self.add_subsystem('comp1', MyComp1())
        self.add_subsystem('comp2', MyComp2())

    def configure(self):
        # splits input via promotes using src_indices
        self.promotes('comp1', inputs=['x'], src_indices=[0, 1, 2])
        self.promotes('comp2', inputs=['x'], src_indices=[3, 4])

p = om.Problem()

p.model.set_input_defaults('x', np.ones(5))
p.model.add_subsystem('G1', MyGroup(), promotes_inputs=['x'])

p.setup()
inp = np.random.random(5)
p.set_val('x', inp)
p.run_model()

print(p.get_val('G1.comp1.x'))
print(p.get_val('G1.comp2.x'))
print(p.get_val('G1.comp1.y'))
print(p.get_val('G1.comp2.y'))

[0.03205746 0.29239725 0.36208276]
[0.96858973 0.42584458]
[1.37307492]
[5.57773722]


In [25]:
assert_near_equal(p.get_val('G1.comp1.x'), inp[:3])
assert_near_equal(p.get_val('G1.comp2.x'), inp[3:])
assert_near_equal(p.get_val('G1.comp1.y'), np.sum(inp[:3]*2))
assert_near_equal(p.get_val('G1.comp2.y'), np.sum(inp[3:]*4))

0.0

```{note}
If the source array is flat, we allow the use of flat src_indices even
without setting `flat_src_indices=True`.
```